In [ ]:
# Download a sample image to test the inference
!wget -O cat.jpg https://upload.wikimedia.org/wikipedia/commons/3/3a/Cat03.jpg
# DOwnload imagenet labels json
!wget - O imagenet_labels.json https: // raw.githubusercontent.com/anishathalye/imagenet-simple-labels/master/imagenet-simple-labels.json

In [1]:
# Load the labels
import json
with open('imagenet_labels.json') as f:
    labels = json.load(f)

In [1]:
import torch
from torchvision import models
import time

In [2]:
# Getting the model

model = models.inception_v3(weights=True)

/home/venom/miniconda3/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [3]:
# Load the image, resize it and convert it to a tensor and display it
from PIL import Image
from torchvision import transforms

img = Image.open('cat.jpg')
img = transforms.Resize((299, 299))(img)
img = transforms.ToTensor()(img)


## CPU

In [4]:
model.eval()

Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stri

In [5]:
# Running Inference once

start = time.time()
out = model(img.unsqueeze(0))
end = time.time()
print('Inference time: ', end - start)

Inference time:  0.12563204765319824


In [6]:
# Running inference 100 times
start = time.time()
for _ in range(100):
    output = model(img.unsqueeze(0))
end = time.time()
print(f'Time: {end - start} seconds')

Time: 8.394792556762695 seconds


## GPU

In [8]:
# Same in GPU
model = model.cuda()
img = img.cuda()
model.eval()

Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stri

In [10]:
# Single image inference
start = time.time()
out = model(img.unsqueeze(0))
end = time.time()
print('Inference time: ', end - start)

Inference time:  0.023438692092895508


In [11]:

start = time.time()
for _ in range(100):
    output = model(img.unsqueeze(0))
end = time.time()
print(f'Time: {end - start} seconds')


Time: 1.2342636585235596 seconds


## Inference Results

In [13]:
# Getting the top 5 predictions
_, indices = torch.sort(out, descending=True)
[(labels[idx], out[0, idx].item()) for idx in indices[0][:5]]


[('Egyptian Mau', 14.871809959411621),
 ('tiger cat', 14.212199211120605),
 ('tabby cat', 13.434951782226562),
 ('lynx', 12.389911651611328),
 ('carton', 8.164073944091797)]

## Conclusion

Its a known fact that GPUs make AI inference faster. In this case, it gave about 3 - 4x speedup. But we can go more!

See you in the next notebook :)
